In [30]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# ------------------ 中文字体设置 ------------------
plt.rcParams['font.sans-serif'] = ['PingFang SC', 'Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# ------------------ 配置路径和文件名 ------------------
folder_path = r"C:\Users\yuanyuan.wang\Desktop\周报"
last_folder_path = r"C:\Users\yuanyuan.wang\Desktop\周报\Week20"
current_folder_path = r"C:\Users\yuanyuan.wang\Desktop\周报\Week21"

last_week_files = [f for f in os.listdir(last_folder_path) if f.endswith('.xlsx')]
current_week_files = [f for f in os.listdir(current_folder_path) if f.endswith('.xlsx')]

# ------------------ 类别配置 ------------------
category_configs = {
    "多头类": {
        "keywords": ["altmultiloanbehaviora", "altmultiloantelcoa", "altmultiplatformbehaviorproa"],
        "rename_map": {
            "altmultiloanbehaviora": "BehaviorA",
            "altmultiloantelcoa": "TelcoA",
            "altmultiplatformbehaviorproa": "BehaviorProA"
        },
        "file_prefix": "多头产品"
    },
    "特殊名单类": {
        "keywords": ["speciallista", "speciallistb", "speciallistb_se", "specialliste", "speciallistpro_ng", "speciallist_mx", "speciallist_ph"],
        "rename_map": {
            "speciallista": "SpecialistA",
            "speciallistb": "SpecialistB",
            "speciallistb_se": "SpecialistB_se",
            "specialliste": "SpecialistE",
            "speciallistpro_ng": "SpecialistPro_NG"
        },
        "file_prefix": "特殊名单产品"
    },
    "客制类": {
        "keywords": ["cust"],
        "rename_map": {},
        "file_prefix": "客制化产品"
    },
    "评分类": {
        "keywords": ["scoregen"],
        "rename_map": {},
        "file_prefix": "通用评分产品"
    },
    "验证类": {
        "keywords": ["phone", "idcheck", "bankcheck"],
        "rename_map": {},
        "file_prefix": "验证类产品"
    },
    "设备类": {
        "keywords": ["equipment", "scorecust"],
        "rename_map": {},
        "file_prefix": "设备类产品"
    },
    "eKYC类": {
        "keywords": ["facecomparison", "idrecognition", "idrecognitionpro", "livenessdetection"],
        "rename_map": {},
        "file_prefix": "eKYC产品"
    }
}

# 映射分类字段
product_type_map = {
    "设备类": "Device anti-fraud", "eKYC类": "ekyc", "评分类": "data product", 
    "客制类": "customized score", "验证类": "data product", "客制类": "data product",
    "特殊名单类": "data product", "多头类": "data product", "评分类": "data product"
}

region_map = {
    "Thailand": "东南亚", "Philippines": "东南亚", "Ghana": "非洲", "Mexico": "拉美",
    "Indonesia": "东南亚", "Nigeria": "非洲", "Kenya": "非洲", "Vietnam": "越南"
}

# ------------------ 日期格式化函数 ------------------
def format_chinese_date(x):
    if isinstance(x, pd.Timestamp):
        return f"{x.month}月{x.day}日"
    return x

# ------------------ 配置函数 ------------------
def map_product_category(product_code, category_configs):
    for category_name, config in category_configs.items():
        for keyword in config["keywords"]:
            if keyword in product_code.lower():
                return category_name
    return "未知"

# ------------------ 主处理函数 ------------------
def process_category(category, config):
    # 处理本周数据
    daily_data_current = {}
    for file in current_week_files:
        date_str = file.split("_")[2]
        file_path = os.path.join(current_folder_path, file)
        df = pd.read_excel(file_path).iloc[:-1]
        df["Product Code Lower"] = df["Product Code"].str.lower()
        filtered_df = df[df["Product Code Lower"].str.contains("|".join(config["keywords"]))]
        grouped = filtered_df.groupby("Product Code")["Effective Request Count"].sum()
        daily_data_current[date_str] = grouped

    df_trend_current = pd.DataFrame(daily_data_current).T.fillna(0)
    if df_trend_current.empty:
        print(f"⚠️ 分类 {category} 没有本周数据，跳过")
        return
    
    df_trend_current.index = pd.to_datetime(df_trend_current.index, format="%Y%m%d")
    df_trend_current = df_trend_current.sort_index()
    
    # 处理上周数据
    daily_data_last = {}
    for file in last_week_files:
        date_str = file.split("_")[2]
        file_path = os.path.join(last_folder_path, file)
        df = pd.read_excel(file_path).iloc[:-1]
        df["Product Code Lower"] = df["Product Code"].str.lower()
        filtered_df = df[df["Product Code Lower"].str.contains("|".join(config["keywords"]))]
        grouped = filtered_df.groupby("Product Code")["Effective Request Count"].sum()
        daily_data_last[date_str] = grouped

    df_trend_last = pd.DataFrame(daily_data_last).T.fillna(0)
    if not df_trend_last.empty:
        df_trend_last.index = pd.to_datetime(df_trend_last.index, format="%Y%m%d")
        df_trend_last = df_trend_last.sort_index()
    
    # 合并本周和上周数据
    df_trend = df_trend_current.copy()
    df_trend.loc["本周总调用量"] = df_trend_current.sum()
    df_trend.loc["日均调用"] = df_trend_current.mean()
    
    # 计算上周总调用量
    last_week_total = df_trend_last.sum().sum() if not df_trend_last.empty else 0
    df_trend.loc["上周总调用量"] = df_trend_last.sum()
    
    # 计算涨幅
    current_week_total = df_trend_current.sum().sum()
#     growth_rate = (current_week_total - df_trend_last.sum()) / df_trend_last.sum() if last_week_total != 0 else 0
    df_trend.loc["涨幅"] = ((df_trend.loc["本周总调用量"]-df_trend.loc["上周总调用量"])/df_trend.loc["上周总调用量"])
    
    # 转置并格式化
#     df_output = df_trend.T.reset_index()
    # 修复KeyError: '产品' - 确保列名正确
    df_output = df_trend.T.rename_axis("产品").reset_index()
    df_output.columns = ["产品"] + [format_chinese_date(c) for c in df_output.columns[1:]]
#     df_output = df_output.rename(columns={"index": "产品"})
    
    # 重命名产品
    if config["rename_map"]:
        df_output["产品"] = df_output["产品"].map(config["rename_map"]).fillna(df_output["产品"])
    
    # 添加总计行
    total_row = {"产品": "总计"}
    for col in df_output.columns[1:]:
        if col in ["涨幅", "日均调用"]:
            if col == "涨幅":
                total_value = (current_week_total - last_week_total) / last_week_total if last_week_total != 0 else 0
            else:
                total_value = df_output[col].iloc[:-4].mean()
        else:
            total_value = df_output[col].sum()
        total_row[col] = total_value
    
    df_output = pd.concat([df_output, pd.DataFrame([total_row])], ignore_index=True)
    
    # 格式化日期列
    date_cols = [col for col in df_output.columns if isinstance(col, pd.Timestamp)]
    for col in date_cols:
        df_output[col] = df_output[col].apply(lambda x: f"{int(x):,}" if pd.notnull(x) else "")
    
    # 格式化涨幅列
    if "涨幅" in df_output.columns:
        df_output["涨幅"] = df_output["涨幅"].apply(
            lambda x: f"{x:.2%}" if isinstance(x, float) else x
        )
    
    # 保存Excel
    excel_path = os.path.join(folder_path, f"{config['file_prefix']}调用趋势表.xlsx")
    df_output.to_excel(excel_path, index=False)
    
    # 绘图（仅本周数据）
    df_plot = df_trend_current.copy()
    plt.figure(figsize=(18, 5))
    for col in df_plot.columns:
        plt.plot(df_plot.index, df_plot[col], marker='o', label=col)
        for i, val in enumerate(df_plot[col]):
            plt.text(df_plot.index[i], val + 30, f"{int(val):,}", ha='center', va='bottom', fontsize=7, alpha=0.8)
    
    plt.title(f"{category}调用趋势", fontsize=16)
    plt.xlabel("日期", fontsize=12)
    plt.ylabel("调用次数", fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend(loc="upper right")
    plt.tight_layout()
    
    image_path = os.path.join(folder_path, f"{config['file_prefix']}调用趋势图.png")
    plt.savefig(image_path, dpi=300)
    plt.close()
    
    print(f"✅ {category} Excel 保存于：{excel_path}")
    print(f"✅ {category} 图表保存于：{image_path}")

# ------------------ 批量处理所有分类 ------------------
for category, config in category_configs.items():
    process_category(category, config)

# ------------------ 周数据读取 ------------------
def read_week_data(files, folder, week_label):
    dfs = []
    for file in files:
        df = pd.read_excel(os.path.join(folder, file)).iloc[:-1]
        df["Week"] = week_label
        dfs.append(df)
    return pd.concat(dfs)

last_week_raw = read_week_data(last_week_files, last_folder_path, "上周")
current_week_raw = read_week_data(current_week_files, current_folder_path, "本周")
combined = pd.concat([last_week_raw, current_week_raw])

# ------------------ 客户聚合 ------------------
customer_agg = combined.groupby(["Short Name", "Week"]).agg(
    {"Effective Request Count": "sum"}
).unstack().fillna(0)
customer_agg.columns = customer_agg.columns.droplevel()
customer_agg = customer_agg.rename(columns={"上周": "上周总调用", "本周": "本周总调用"})
customer_agg["环比"] = (
    (customer_agg["本周总调用"] - customer_agg["上周总调用"]) / 
    customer_agg["上周总调用"].replace(0, np.nan)
).fillna(0).apply(lambda x: f"{x:.2%}")

extras = combined.groupby("Short Name").agg({
    "Account ID": lambda x: list(sorted(set(x))),
    "Product Code": lambda x: list(sorted(set(x))),
    "Country": lambda x: list(sorted(set(x)))
})
customer_summary = pd.concat([customer_agg, extras], axis=1).reset_index()

# ------------------ 账户聚合 ------------------
account_agg = combined.groupby(
    ["Account ID", "Short Name", "Country", "Week"]
)["Effective Request Count"].sum().unstack().fillna(0)
account_agg = account_agg.rename(columns={"上周": "上周总调用", "本周": "本周总调用"})
account_agg["环比"] = (
    (account_agg["本周总调用"] - account_agg["上周总调用"]) / 
    account_agg["上周总调用"].replace(0, np.nan)
).fillna(0).apply(lambda x: f"{x:.2%}")
account_summary = account_agg.reset_index()

# ------------------ 产品调用变化分析 ------------------
def read_and_merge(files, folder):
    dfs = []
    for f in files:
        df = pd.read_excel(os.path.join(folder, f)).iloc[:-1]
        dfs.append(df)
    merged = pd.concat(dfs)
    return merged.groupby(
        ["Product Code", "Short Name", "Country"], as_index=False
    )["Effective Request Count"].sum()

last_week_df = read_and_merge(last_week_files, last_folder_path)
current_week_df = read_and_merge(current_week_files, current_folder_path)

comparison = pd.merge(
    current_week_df, last_week_df, 
    on=["Product Code", "Short Name", "Country"], 
    suffixes=("_current_week", "_last_week"), 
    how="outer"
).fillna(0)

comparison["变化量"] = (
    comparison["Effective Request Count_current_week"] - 
    comparison["Effective Request Count_last_week"]
).astype(int)

comparison["涨幅"] = (
    (comparison["变化量"] / comparison["Effective Request Count_last_week"].replace(0, np.nan))
    .fillna(0).apply(lambda x: f"{x:.2%}")
)

comparison["产品分类"] = comparison["Product Code"].apply(
    lambda x: map_product_category(x, category_configs)
)

# 确保有数据再继续
if not comparison.empty:
    top_increase = comparison.loc[comparison.groupby("产品分类")["变化量"].idxmax()]
    top_decrease = comparison.loc[comparison.groupby("产品分类")["变化量"].idxmin()]
    final_result = pd.concat([
        top_increase.assign(变化类型="增幅最大"), 
        top_decrease.assign(变化类型="降幅最大")
    ]).reset_index(drop=True)

    final_result = final_result[[
        "产品分类", "变化类型", "Product Code", "Short Name", "Country", "变化量", "涨幅"
    ]]
else:
    final_result = pd.DataFrame(columns=[
        "产品分类", "变化类型", "Product Code", "Short Name", "Country", "变化量", "涨幅"
    ])

# ------------------ 底层明细合并与指标计算 ------------------
def merge_daily_data(files, folder):
    merged_df = None
    for file in files:
        df = pd.read_excel(os.path.join(folder, file)).iloc[:-1]
        date_suffix = os.path.splitext(file)[0][-4:]
        df.rename(columns={"Effective Request Count": date_suffix}, inplace=True)
        df_subset = df[["Product Code", "Product Version", "Account ID", "Country", "Short Name", date_suffix]]
        if merged_df is None:
            merged_df = df_subset
        else:
            merged_df = pd.merge(merged_df, df_subset, how='outer', 
                                on=["Product Code", "Product Version", "Account ID", "Country", "Short Name"])
    return merged_df

merged_df_last = merge_daily_data(last_week_files, last_folder_path)
merged_df_current = merge_daily_data(current_week_files, current_folder_path)

# 计算指标
date_cols_current = []
if not merged_df_current.empty:
    date_cols_current = [col for col in merged_df_current.columns if col.isdigit()]
    merged_df_current["本周总调用量"] = merged_df_current[date_cols_current].sum(axis=1)
    merged_df_current["日均调用量"] = round(merged_df_current["本周总调用量"] / len(date_cols_current), 0)
    merged_df_current["单日最大调用量"] = merged_df_current[date_cols_current].max(axis=1)
    merged_df_current["单日最小调用量"] = merged_df_current[date_cols_current].min(axis=1)
else:
    merged_df_current = pd.DataFrame(columns=[
        "Product Code", "Product Version", "Account ID", "Country", "Short Name", 
        "本周总调用量", "日均调用量", "单日最大调用量", "单日最小调用量"
    ])

date_cols_last = []
if not merged_df_last.empty:
    date_cols_last = [col for col in merged_df_last.columns if col.isdigit()]
    merged_df_last["上周总调用量"] = merged_df_last[date_cols_last].sum(axis=1)
else:
    merged_df_last = pd.DataFrame(columns=[
        "Product Code", "Product Version", "Account ID", "Country", "Short Name", "上周总调用量"
    ])

# 合并上周和本周数据
if not merged_df_current.empty and not merged_df_last.empty:
    merged_df = pd.merge(
        merged_df_current, 
        merged_df_last[["Product Code", "Product Version", "Account ID", "Country", "Short Name", "上周总调用量"]],
        on=["Product Code", "Product Version", "Account ID", "Country", "Short Name"],
        how="left"
    ).fillna(0)
elif not merged_df_current.empty:
    merged_df = merged_df_current.copy()
    merged_df["上周总调用量"] = 0
else:
    merged_df = pd.DataFrame(columns=[
        "Product Code", "Product Version", "Account ID", "Country", "Short Name", 
        "本周总调用量", "日均调用量", "单日最大调用量", "单日最小调用量", "上周总调用量"
    ])

# 计算环比增幅
if not merged_df.empty:
    merged_df["环比增幅"] = (
        (merged_df["本周总调用量"] - merged_df["上周总调用量"]) / 
        merged_df["上周总调用量"].replace(0, np.nan)
    ).fillna(0).apply(lambda x: f"{x:.2%}")
else:
    merged_df["环比增幅"] = ""

# 添加分类信息
if not merged_df.empty:
    merged_df["产品分类"] = merged_df["Product Code"].apply(
        lambda x: map_product_category(x, category_configs)
    )
    merged_df["Type"] = merged_df["产品分类"].map(product_type_map)
    merged_df["地区"] = merged_df["Country"].map(region_map)
else:
    merged_df["产品分类"] = ""
    merged_df["Type"] = ""
    merged_df["地区"] = ""

# 排序字段
final_cols = ["Type", "产品分类", "地区", "Product Code", "Product Version", "Account ID", 
              "Country", "Short Name"] + date_cols_current + [
    "日均调用量", "单日最大调用量", "单日最小调用量", "本周总调用量", 
    "上周总调用量", "环比增幅"
]
merged_df = merged_df[final_cols] if not merged_df.empty else pd.DataFrame(columns=final_cols)

# ------------------ 输出数据 ------------------
customer_summary.to_excel(os.path.join(folder_path, "客户聚合统计_最终.xlsx"), index=False)
account_summary.to_excel(os.path.join(folder_path, "账户级统计_最终.xlsx"), index=False)
final_result.to_excel(os.path.join(folder_path, "产品调用变化明细.xlsx"), index=False)
merged_df.to_excel(os.path.join(folder_path, "周报_最终底表.xlsx"), index=False)
print("✅ 三张统计表+底表已成功导出！")

# ------------------ 饼图绘制 ------------------
def draw_labeled_pie(df, group_col, value_col, title, filename):
    if df.empty:
        print(f"⚠️ 没有数据绘制饼图: {title}")
        return
        
    stat = df.groupby(group_col)[value_col].sum().sort_values(ascending=False)
    labels = stat.index.tolist()
    values = stat.values
    total = values.sum()
    
    if total == 0:
        print(f"⚠️ 没有有效数据绘制饼图: {title}")
        return
        
    display_labels = [f"{l}, {v / total:.2%}" for l, v in zip(labels, values)]
    
    fig, ax = plt.subplots(figsize=(8, 6))
    wedges, _ = ax.pie(
        values,
        labels=None,
        startangle=90,
        colors=plt.cm.tab20.colors,
        wedgeprops=dict(width=0.6)
    )
    
    for i, (wedge, label) in enumerate(zip(wedges, display_labels)):
        angle = (wedge.theta2 + wedge.theta1) / 2
        x = np.cos(np.radians(angle)) * 0.85
        y = np.sin(np.radians(angle)) * 0.85
        ax.text(x, y, label, ha='center', va='center', fontsize=10, weight='bold')
    
    plt.title(title, fontsize=16)
    plt.legend(wedges, display_labels, title=group_col, loc='center left', bbox_to_anchor=(1, 0.5))
    ax.axis('equal')
    plt.tight_layout()
    out_path = os.path.join(folder_path, filename)
    plt.savefig(out_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"📊 饼图导出成功：{filename}")

# ------------------ 执行绘图 ------------------
if not merged_df.empty:
    draw_labeled_pie(merged_df, "产品分类", "本周总调用量", "调用产品占比", "调用产品占比.png")
    draw_labeled_pie(merged_df, "Short Name", "本周总调用量", "客户调用占比", "客户调用占比.png")
    draw_labeled_pie(merged_df, "地区", "本周总调用量", "地区调用占比", "地区调用占比.png")
    draw_labeled_pie(merged_df, "Country", "本周总调用量", "国家调用占比", "国家调用占比.png")
else:
    print("⚠️ 没有数据绘制饼图")

# ------------------ 客户数聚合 ------------------
def classify_and_sum(df, category_configs):
    if df.empty:
        return pd.Series(dtype=float)
    
    df["产品分类"] = df["Product Code"].apply(lambda x: map_product_category(x, category_configs))
    df = df.dropna(subset=["产品分类"])
    return df.groupby("产品分类")["Effective Request Count"].sum()

if not last_week_df.empty:
    last_week_df["产品分类"] = last_week_df["Product Code"].apply(
        lambda x: map_product_category(x, category_configs))
if not current_week_df.empty:
    current_week_df["产品分类"] = current_week_df["Product Code"].apply(
        lambda x: map_product_category(x, category_configs))

# 按分类统计客户数（去重）
if not last_week_df.empty:
    last_week_client_count = last_week_df[["产品分类", "Short Name"]].drop_duplicates().groupby("产品分类").size().rename("上周客户数")
else:
    last_week_client_count = pd.Series(dtype=int).rename("上周客户数")

if not current_week_df.empty:
    current_week_client_count = current_week_df[["产品分类", "Short Name"]].drop_duplicates().groupby("产品分类").size().rename("本周客户数")
else:
    current_week_client_count = pd.Series(dtype=int).rename("本周客户数")

client_agg = pd.concat([last_week_client_count, current_week_client_count], axis=1).fillna(0).astype(int)

# 调用次数聚合
last_week_calls = classify_and_sum(last_week_df.copy(), category_configs)
current_week_calls = classify_and_sum(current_week_df.copy(), category_configs)

# 获取客户列表
def get_clients_by_category(df):
    if df.empty:
        return {}
    return df.groupby("产品分类")["Short Name"].agg(lambda x: set(x.unique())).to_dict()

last_week_clients_by_cat = get_clients_by_category(last_week_df)
current_week_clients_by_cat = get_clients_by_category(current_week_df)

# 合并客户数与调用次数
full_df = client_agg.copy()
full_df["上周调用"] = last_week_calls
full_df["本周调用"] = current_week_calls
full_df = full_df.fillna(0).astype(int)

# 添加合计
if not last_week_df.empty:
    full_df.loc["合计", "上周调用"] = last_week_df["Effective Request Count"].sum()
if not current_week_df.empty:
    full_df.loc["合计", "本周调用"] = current_week_df["Effective Request Count"].sum()

# 计算占比和变化
if not current_week_df.empty:
    total_current_calls = current_week_df["Effective Request Count"].sum()
    full_df["占比"] = (full_df["本周调用"] / total_current_calls).apply(lambda x: f"{x:.0%}")
else:
    full_df["占比"] = "0%"

full_df["变化"] = full_df["本周调用"] - full_df["上周调用"]
full_df["较上周相对涨幅"] = (full_df["变化"] / full_df["上周调用"].replace(0, 1)).apply(lambda x: f"{x:.2%}{'↑' if x > 0 else ('↓' if x < 0 else '')}")

# 变化客户
Change_customer = []
for cat in full_df.index:
    if cat == "合计":
        Change_customer.append("")
        continue
    
    last_clients = last_week_clients_by_cat.get(cat, set())
    curr_clients = current_week_clients_by_cat.get(cat, set())
    
    new_clients = curr_clients - last_clients
    lost_clients = last_clients - curr_clients
    
    parts = []
    if new_clients:
        parts.append("新增：" + "、".join(sorted(new_clients)))
    if lost_clients:
        parts.append("流失：" + "、".join(sorted(lost_clients)))
    
    Change_customer.append(" | ".join(parts) if parts else "")

full_df["变化客户"] = Change_customer

# 输出最终结果
final_df = full_df.reset_index()[[
    "产品分类", "上周客户数", "本周客户数", "变化客户",
    "上周调用", "本周调用", "占比", "变化", "较上周相对涨幅"
]]

# 保存结果
final_df.to_excel(os.path.join(folder_path, "调用客户与调用量统计.xlsx"), index=False)
print("✅ 客户对比表格已完成！")

✅ 多头类 Excel 保存于：C:\Users\yuanyuan.wang\Desktop\周报\多头产品调用趋势表.xlsx
✅ 多头类 图表保存于：C:\Users\yuanyuan.wang\Desktop\周报\多头产品调用趋势图.png
✅ 特殊名单类 Excel 保存于：C:\Users\yuanyuan.wang\Desktop\周报\特殊名单产品调用趋势表.xlsx
✅ 特殊名单类 图表保存于：C:\Users\yuanyuan.wang\Desktop\周报\特殊名单产品调用趋势图.png
✅ 客制类 Excel 保存于：C:\Users\yuanyuan.wang\Desktop\周报\客制化产品调用趋势表.xlsx
✅ 客制类 图表保存于：C:\Users\yuanyuan.wang\Desktop\周报\客制化产品调用趋势图.png
✅ 评分类 Excel 保存于：C:\Users\yuanyuan.wang\Desktop\周报\通用评分产品调用趋势表.xlsx
✅ 评分类 图表保存于：C:\Users\yuanyuan.wang\Desktop\周报\通用评分产品调用趋势图.png
✅ 验证类 Excel 保存于：C:\Users\yuanyuan.wang\Desktop\周报\验证类产品调用趋势表.xlsx
✅ 验证类 图表保存于：C:\Users\yuanyuan.wang\Desktop\周报\验证类产品调用趋势图.png
✅ 设备类 Excel 保存于：C:\Users\yuanyuan.wang\Desktop\周报\设备类产品调用趋势表.xlsx
✅ 设备类 图表保存于：C:\Users\yuanyuan.wang\Desktop\周报\设备类产品调用趋势图.png
✅ eKYC类 Excel 保存于：C:\Users\yuanyuan.wang\Desktop\周报\eKYC产品调用趋势表.xlsx
✅ eKYC类 图表保存于：C:\Users\yuanyuan.wang\Desktop\周报\eKYC产品调用趋势图.png
✅ 三张统计表+底表已成功导出！
📊 饼图导出成功：调用产品占比.png
📊 饼图导出成功：客户调用占比.png
📊 饼图导出成功：地区调用占比.png
📊 饼图导出成功：国家调用占比.png
✅ 